In [6]:
from headers import *
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, make_scorer, confusion_matrix
from sklearn.pipeline import Pipeline
from joblib import dump, load
from IPython.display import Markdown, display
from math import sqrt

In [7]:
df = pd.read_csv('../data/combined_inner.csv')
X = df[ratioKeys + relativeRatioKeys]

for yLabel in yAlpha:
    yContinuous = df[yLabel]
    y = np.where(yContinuous <= 0, 0, 1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2, stratify=y)
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('tree', DecisionTreeClassifier())
    ])

    param_grid = {
        'tree__criterion': ["gini", "entropy", "log_loss"],
    }

    grid = GridSearchCV(pipeline,
                        param_grid,
                        cv=StratifiedKFold(3),
                        verbose=1,
                        refit=True,
                        n_jobs=-1
                        )
    grid.fit(X_train, y_train)
    y_pred = grid.predict(X_test)
    xTaken = X_test[y_pred == 1]
    wholeTestData = df.loc[df.index.intersection(X_test.index)]
    wholeTestData.to_csv('../data/tree_results/' + yLabel + '.csv', index=False)

    takenData = df.loc[df.index.intersection(xTaken.index)]

    estimator = grid.best_estimator_
    dump(grid, '../data/tree_models/' + yLabel + '.joblib')

    print(f'Results {yLabel} mean of: {takenData.loc[:, yLabel].mean()}')

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Results alpha1Year mean of: 0.15653675508716436
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Results adjustedAlpha1Year mean of: 0.1635819123795836
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Results equalAlpha1Year mean of: 0.1441426871690846
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Results equalAdjustedAlpha1Year mean of: 0.15035732547406913
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Results alpha2Year mean of: 0.32320891221244263
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Results adjustedAlpha2Year mean of: 0.28812714479347945
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Results equalAlpha2Year mean of: 0.31628891587900254
Fitting 3 folds for each of 3 candidates, totalling 9 fits
Results equalAdjustedAlpha2Year mean of: 0.3211670876589014


In [8]:
for label in yAlpha:
    model = load('../data/tree_models/' + label + '.joblib')
    testedData = pd.read_csv('../data/tree_results/' + label + '.csv')

    X = testedData[ratioKeys + relativeRatioKeys]
    y = np.where(testedData[label] <= 0, 0, 1)
    display(Markdown(f'**Results for key: {label}**'))
    print(f'data count: {testedData[label].count()}')
    print(f'data mean: {testedData[label].mean()}')
    print(f'data median: {testedData[label].median()}')

    print('====TEST=====')
    y_pred = model.predict(X)
    xTaken = X[y_pred == 1]
    results = testedData.loc[testedData.index.intersection(xTaken.index)]

    print(f'res count {results[label].count()}')
    print(f'res mean {results[label].mean()}')
    print(f'res median {results[label].median()}')

    print(confusion_matrix(y, y_pred))

    display(Markdown(f'**diff: {results[label].mean() - testedData[label].mean()}**'))
    if '2' in label:
        cagrToBenchmark = sqrt(results[label].mean() - testedData[label].mean() + 1) - 1
        print(f'CAGR: {cagrToBenchmark}')
    print('-------------------------------------------------')

# TODO: iterate over y cutoffs. Train each cutoff with gridCV. Train linearSVC first, and check how faster it is. 
# TODO: maybe use (Linear)SVR - regression, since we have linear regression

**Results for key: alpha1Year**

data count: 6053
data mean: 0.0300172769962029
data median: -0.0031354597826141
====TEST=====
res count 3052
res mean 0.15653675508716436
res median 0.1198777801275849
[[2269  788]
 [ 732 2264]]


**diff: 0.12651947809096145**

-------------------------------------------------


**Results for key: adjustedAlpha1Year**

...

CAGR: 0.15028889404801649
-------------------------------------------------


**Results for key: equalAdjustedAlpha2Year**

data count: 6053
data mean: 0.002947332607081462
data median: -0.0894219371820089
====TEST=====
res count 2438
res mean 0.3211670876589014
res median 0.1774244201179481
[[3003  596]
 [ 612 1842]]


**diff: 0.31821975505181993**

CAGR: 0.14813751574095857
-------------------------------------------------
